In [2]:
import midynet
import midynet.util.display as display
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import pathlib

display.setup_dir("./")

plot = midynet.metrics.Statistics.plot

paths_to_figures = [".", "../../../../papers/p-midynet/paper1-mutual_info/figures"]
paths_to_figures = list(map(pathlib.Path, paths_to_figures))
path_to_data = pathlib.Path("../../../../data/midynet-data/figure4")

In [5]:
exp = {}
hg, hx, hgx, hxg, mi = {}, {}, {}, {}, {}
T = 10

all_dynamics = ["ising", "sis", "cowan"]
for dynamics in all_dynamics:
    path = path_to_data / f"figure4-nbinom-{dynamics}.num_steps{T}"
    if (path / "config.pickle").exists():
        exp[dynamics] = midynet.experiments.Experiment.load_from_file(path / "config.pickle")
        hg[dynamics] = midynet.metrics.Statistics({k: exp[dynamics].metrics["mutualinfo"].data[exp[dynamics].name][f"hg-{k}"] for k in ["mid", "high", "low"]})
        hx[dynamics] = midynet.metrics.Statistics({k: exp[dynamics].metrics["mutualinfo"].data[exp[dynamics].name][f"hx-{k}"] for k in ["mid", "high", "low"]})
        hgx[dynamics] = midynet.metrics.Statistics({k: exp[dynamics].metrics["mutualinfo"].data[exp[dynamics].name][f"hgx-{k}"] for k in ["mid", "high", "low"]})
        hxg[dynamics] = midynet.metrics.Statistics({k: exp[dynamics].metrics["mutualinfo"].data[exp[dynamics].name][f"hxg-{k}"] for k in ["mid", "high", "low"]})
        mi[dynamics] = midynet.metrics.Statistics({k: exp[dynamics].metrics["mutualinfo"].data[exp[dynamics].name][f"mi-{k}"] for k in ["mid", "high", "low"]})

xlabel = r"Number of time steps"
titles = {"ising": r"Glauber", "sis": r"SIS", "cowan": r"Cowan"}
couplingLabels = {"ising": r"J\langle k \rangle", "sis": r"\lambda \langle k \rangle", "cowan": r"\nu\langle k \rangle"}

In [15]:
hx["cowan"]

Statistics(mid=[[-1.12200536e+02 -9.32837603e+01 -8.91675600e+01 -8.54042852e+01
  -8.09474297e+01 -3.46244606e+01 -2.81752793e+01 -2.95872136e+01
  -2.86241587e+01 -1.34759625e+01 -1.69355243e+01 -2.10122344e+01
  -8.49264460e+00 -2.53755720e+00  7.64121759e-01  4.98669513e+00
  -9.25636725e-01  1.44285832e+01  4.83805038e+00  1.74478168e+01]
 [-1.12248245e+02 -9.28632806e+01 -8.89209496e+01 -8.52244266e+01
  -8.06931429e+01 -3.41526813e+01 -2.81864166e+01 -2.73364729e+01
  -2.66823985e+01 -1.29041873e+01 -1.52937281e+01 -1.55278749e+01
  -6.60527329e+00 -1.68980938e+00  2.51312752e+00  6.81531617e+00
  -6.13433252e-01  1.47534783e+01  7.18886869e+00  1.92142686e+01]
 [-1.06957151e+02 -8.50733413e+01 -7.87397986e+01 -7.50548552e+01
  -7.08653267e+01 -2.32704707e+01 -1.07613193e+01 -1.65322636e+01
  -1.24057201e+01  4.99898013e-01 -2.24266684e+00 -3.75196934e+00
   1.06838795e+01  1.81065291e+01  1.61876824e+01  2.67729998e+01
   1.70821606e+01  3.44137149e+01  2.25061002e+01  4.366968